# Data-Parallel to Distributed Data-Parallel

In this session, we're going to try and bridge the gap between data parallelism in the shared memory case, which is what we'd learned in the parallel programming course and distributed data parallelism.
So taking that idea of data parallelism and extending that to the situation where you no longer have data on just one node anymore. Now you have data spread across several independent nodes. 

Scala's Parallel Collections is a collections abstraction over shared memory data-parallel execution.

Shared memory data parallelism: 

- Split the data on same machine. 
- Workers/threads independently operate on the data shards in parallel. 
- Combine when done (if necessary).

Distributed Data-Parallelism

- Split the data over several nodes. 
- Nodes independently operate on the data shards in parallel. 
- Combine when done ( if necessary).

We have to worry about network latency because of data sharing and communication between nodes.

However, like parallel collections, we can keep  collections  abstraction over distributed data-parallel execution.

> **Shared  memory case**: 
 Data-parallel programming model. Data partitioned in memory and operated upon in parallel. 

> **Distributed case**:  
Data-parallel programming model. Data partitioned between machines, network in between, operated upon in parallel. 

Overall, most all properties we learned about related to shared memory data-parallel collections can be applied to their distributed counterparts. 
However, must now consider latency when using our model. 

Throughout this part of the course we will use the Apache Spark framework for distributed data-parallel programming. 

Spark implements a distributed data parallel model called Resilient Distributed Datasets (RDDs)

RDD are distributed counterparts of parallel collections.

## Distributed Data-Parallel: High Level Illustration 
Given a large datset (say Wikiepdia English 48.4GB) that can't fit into memory of single node. SPark will chunk up the data using some distribution mechanism and distribute it over cluster of machines.

From there think of your distributed data like single collection. Spark will return a reference to entire distributed datasets.

```scala
val wiki: RDD[WikiArticle] = ...
wiki.map { article=> article.text.toLowerCase } 
```

# Latency

So before we get into how to use Spark, how to get good performance out of Spark,
and how to express basic analytics jobs in Spark's programming model, let's first
look at some of the key ideas behind Spark in an effort to get a bit of intuition
about why Spark is causing such a shift in the world of data science and analytics.
As we'll see in this section, Spark stands out in how it deals with latency,
which is a fundamental concern when a system becomes distributed. 

__Data-Parallel Programming In the Parallel Programming course, we learned:__
- Data parallelism on a single multicore/multi-processor machine.
- Parallel collections as an implementation of this paradigm. 

__Today:__
- Data parallelism in a distributed setting.
- Distributed collections abstraction from Apache Spark as an implementation of this paradigm.

Distribution introduces important concerns beyond what we had to worry about when dealing with parallelism in the shared memory case: 

1. Partial failure: crash failures of a subset of the machines involved in a distributed computation. 
2. Latency: certain operations have a much higher latency than other operations due to network communication.

Memory Ops < Disk Ops < Network Comms 

## Big  Data Processing and Latency 

With some intuition now about how expensive network communication and disk operations can be, one may ask: 
How do these latency numbers relate to big data processing? 

To answer this question, let's first start with Spark's predecessor, __Hadoop__. 

Hadoop is a widely-used large-scale batch data processing framework.  It's an open source implementation of Google's MapReduce. 
MapReduce was ground-breaking because it provided: 
- a simple API (simple map and reduce steps) -
-  **fault tolerance** 

Fault tolerance is what made it possible for Hadoop/MapReduce to scale to 100s or 1000s of nodes at all.

Hadoop/MapReduce + Fault Tolerance 
Why is this important? 
For  100s or  1000s of old commodity machines, likelihood of at least one node failing is very high midway th rough a job. 
Thus, Hadoop/MapReduce's ability to recover from node failure enabled: computations on unthinkably large data sets to succeed to completion. 

**Fault tolerance + simple API = At Google,  MapReduce made it possible for an average Google software engineer to craft a   complex pipeline of map/reduce stages on extremely large data sets.**


## Why Spark? 

Fault-tolerance in  Hadoop/MapReduce comes at a cost. Between each map and reduce step, in order to recover from potential failures,  Hadoop/MapReduce shuffles its data and write intermediate data to disk. 

> Remember: 
Reading/writing to disk: lOOx slower than in-memory   Network communication: 1,000,000x slower than in-memory 

Spark 

- retains fault-tolerance
- Different strategy for handling latency (latency significantly reduced!) 

Spark uses different strategy to reduce latency. Achieves this using ideas from functional programming! 
> Idea:  Keep all data immutable and in-memory.  All operations on data are just functional transformations, like regular Scala collections.  Fault tolerance is achieved by replaying functional transformations over original dataset.

Spark has been shown to be l00x more performant than Hadoop, while adding even more expressive APls. Spark tries to minimize agressively any network traffic and favours in-memory computation